In [2]:
!pip install mysql-connector-python langchain langchain_openai openai langchain_experimental sqlalchemy pinecone-client -q

You should consider upgrading via the '/Users/danielhernandez/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [56]:
import sqlalchemy
import pandas as pd

database_username = "username"
database_password = "password"
database_ip = "host"
database_name = "name"
database_port = "port"

database_connection = sqlalchemy.create_engine(
    "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
        database_username, database_password, database_ip, database_port, database_name
    )
)

comments = pd.read_sql_query(
    """SELECT * FROM all_star_comment""", con=database_connection
)

comments = comments[['stars', 'comment']]
comments

,stars,comment
0,5,"El servicio en este hotel es excepcional, el p..."
1,5,"Las instalaciones son impecables, todo está mu..."
2,4,La comida en el restaurante del hotel es delic...
3,4,"La ubicación es perfecta, cerca de las princip..."
4,4,"Las habitaciones son cómodas y espaciosas, ide..."
5,5,El Wi-Fi funciona perfectamente en todo el hot...
6,4,"El desayuno buffet es abundante y variado, con..."
7,5,El ambiente del hotel es muy acogedor y tranqu...
8,5,El spa y las áreas de bienestar son excelentes...
9,4,El hotel ofrece una gran cantidad de servicios...


### Best AI Agent (Until Now)

In [57]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

def analyze_business_feedback(query):

    llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

    agent_executor = create_python_agent(
        llm=llm,
        tool=PythonREPLTool(),
        verbose=True,
        agent_executor_kwargs={"handle_parsing_errors": True, "timeout": 60}
    )

    data_description = comments.to_string(index=False)

    prompt = f"""

    Eres un experto analista de retroalimentación de clientes, especializado en identificar patrones y tendencias en la retroalimentación.

    El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente, sino también identificar oportunidades de mejora en la experiencia general del cliente dentro del negocio donde trabajan. Ya que los comentarios de los clientes revelan mucho más que solo un buen servicio al cliente.

    Si no sabes la respuesta a una pregunta, simplete response, no sé.

    Dame una respuesta corta y concreta.

    En caso de "Observation: Invalid Format: Missing 'Action:' after 'Thought: Thought:I don't know", responde:

    Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.

    Deberías usar las herramientas que están abajo para responder la pregunta que te hicieron:

    Intenta responder la siguiente pregunta: {query}, analizando los siguientes datos buscando keywords de la pregunta {data_description} o dando una respuesta que pueda ser relevante.

    """

    response = agent_executor.invoke(prompt)
    return response['output']


analyze_business_feedback('Que debería mejorar de mi servicio?')




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [54]:
import gradio as gr

demo = gr.Interface(fn=analyze_business_feedback, inputs="text", outputs="text")

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------


> Entering new AgentExecutor chain...
I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Final Answer: Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.

> Finished chain.


> Entering new AgentExecutor chain...
I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Final Answer: Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.

> Finished chain.


> Entering n

### Running Agent with Memory

In [55]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize memory store
memory_store = {}

def analyze_business_feedback(query):
    llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

    agent_executor = create_python_agent(
        llm=llm,
        tool=PythonREPLTool(),
        verbose=True,
        agent_executor_kwargs={"handle_parsing_errors": True, "timeout": 60}
    )

    # Convert comments to a string if they are not already
    data_description = memory_store.get('comments', '')

    prompt = f"""
    Eres un experto analista de retroalimentación de clientes, especializado en identificar patrones y tendencias en la retroalimentación.
    El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente, sino también identificar oportunidades de mejora en la experiencia general del cliente dentro del negocio donde trabajan. Ya que los comentarios de los clientes revelan mucho más que solo un buen servicio al cliente.
    Si no sabes la respuesta a una pregunta, simplemente responde, no sé.
    Dame una respuesta corta y concreta.
    En caso de "Observation: Invalid Format: Missing 'Action:' after 'Thought: Thought:I don't know", responde:
    Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.
    Deberías usar las herramientas que están abajo para responder la pregunta que te hicieron:
    Intenta responder la siguiente pregunta: {query}, analizando los siguientes datos buscando keywords de la pregunta {data_description} o dando una respuesta que pueda ser relevante.
    """

    # Execute the prompt and save the response
    response = agent_executor.invoke(prompt)
    memory_store['last_query'] = query
    memory_store['last_response'] = response['output']
    return response['output']

# Example usage
analyze_business_feedback('Que debería mejorar de mi servicio?')




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}